# Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Prepare inputs and targets

In [2]:
facebook_sentiment = pd.read_csv("../data/Twitter_sentiment/facebook_sentiment_time_series.csv")
facebook_sentiment.head()

,date,binary_mean,binary_squared_mean,binary_sqrt_mean,prob_mean,prob_squared_mean,prob_sqrt_mean
0,1/2/2020,-1.000,-1.000,-1.000,-0.364,-0.344,-0.376
1,1/3/2020,-1.000,-1.000,-0.999,-0.479,-0.477,-0.480
2,1/5/2020,0.304,0.272,0.319,0.136,0.086,0.162
3,1/6/2020,-0.372,-0.430,-0.350,-0.388,-0.437,-0.371
4,1/7/2020,-0.280,-0.589,-0.180,0.066,-0.007,0.074


In [3]:
facebook_lstm = pd.read_csv("../data/LSTM/FB_LSTM_predictions.csv")
facebook_lstm.head()

,Unnamed: 0,Date,Pred,Actual
0,1916,1/2/2020,208.330950,209.779999
1,1917,1/3/2020,212.493704,208.669998
2,1918,1/6/2020,211.364467,212.600006
3,1919,1/7/2020,215.023133,213.059998
4,1920,1/8/2020,215.540427,215.220001


In [4]:
merged = pd.merge(facebook_sentiment, facebook_lstm, left_on='date', right_on='Date')

In [5]:
merged_changed = merged[['date', 'prob_sqrt_mean', 'Pred', 'Actual']]
merged_changed = merged_changed.dropna()

In [6]:
inputs_1 = merged_changed.prob_sqrt_mean.values
inputs_1

array([-0.376, -0.48 , -0.371,  0.074, -0.012, -0.566, -0.113, -0.068,
       -0.371, -0.016, -0.045, -0.17 , -0.821,  0.324, -0.206, -0.229,
        0.13 , -0.079, -0.139,  0.443,  0.314, -0.378, -0.204, -0.079,
       -0.035, -0.008,  0.081,  0.204, -0.765,  0.157,  0.114,  0.13 ,
       -0.141, -0.12 ,  0.3  , -0.035, -0.056, -0.468, -0.424, -0.335,
       -0.024,  0.075, -0.331, -0.292, -0.493, -0.417,  0.058,  0.101,
       -0.528, -0.2  ,  0.262,  0.184,  0.084,  0.422, -0.142,  0.097,
        0.653, -0.11 ,  0.121, -0.254,  0.055,  0.716,  0.068, -0.245,
        0.078,  0.187, -0.518,  0.196,  0.165, -0.321, -0.42 , -0.187,
        0.117,  0.245, -0.208,  0.045,  0.407, -0.275, -0.193,  0.316,
       -0.069,  0.005,  0.064,  0.045, -0.073,  0.016, -0.134, -0.121,
        0.051,  0.05 ,  0.253, -0.02 ,  0.073, -0.05 ,  0.112, -0.034,
       -0.013, -0.101, -0.072, -0.152, -0.133, -0.19 , -0.034, -0.083,
        0.042,  0.221, -0.167,  0.025, -0.39 , -0.073, -0.006,  0.148,
      

In [7]:
inputs_2 = merged_changed.Pred.values
inputs_2.shape

(217,)

In [8]:
inputs = np.stack((inputs_1, inputs_2), axis=1)
inputs.shape

(217, 2)

In [9]:
targets = merged_changed.iloc[:, 3].values

# Fit a model

In [10]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
inputs = sc.fit_transform(inputs)

inputs = sm.add_constant(inputs)
model = sm.OLS(targets, inputs)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     3504.
Date:                Mon, 15 Feb 2021   Prob (F-statistic):          3.00e-164
Time:                        22:22:13   Log-Likelihood:                -711.79
No. Observations:                 217   AIC:                             1430.
Df Residuals:                     214   BIC:                             1440.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        229.2478      0.440    521.436      0.0

# Save Results

In [11]:
predictions = results.predict()

In [12]:
facebook_stock_prediction = pd.DataFrame().assign(Date=merged_changed['date'], Predictions=predictions, 
                                                Actual=merged_changed['Actual'])
facebook_stock_prediction.to_csv("../Stock Price Predictions with Twitter/facebook_stock_prediction_twitter.csv")